In [11]:
#사용환경을 준비! 참고로 저는 환경변수에 넣었기 때문에(공유가 아니라 제가 이미 돈을 넣어서 제 계정으로 된 API를 쓰고 있습니다!)
#os.getenv를 통해 제 환경변수에 있는 API키 가져오기! 그리고 불필요한 경고문 제거를 위해 warnings 추가
import os
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")
import warnings
warnings.filterwarnings(action = 'ignore')

In [12]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader

client = ChatOpenAI(model = 'gpt-4o', temperature = 0.5, max_tokens = 2000)
#PyPDFLoader를 통해 pdf 파일 모델 불러오기!
loader = PyPDFLoader('C:\\Users\\kevinkim\\OneDrive\\바탕 화면\\Sparta_Work\\초거대 언어모델 연구 동향.pdf')
docs = loader.load()

In [13]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
#요건 RecursiveCharacterTextSplitter: CharacterTextSplitter와 유사하지만, 더 복잡한 분할 방식을 제공
recursive_text_splitter = RecursiveCharacterTextSplitter(
    #chunk_size: 각 청크의 최대 길이
    chunk_size=500,
    #chunk_overlap: 청크 간 겹침의 길이
    chunk_overlap=10,
    #lenth_function: 텍스트 청크의 길이를 계산하는 데 사용하는 함수(기본적으로 len 함수 사용)
    length_function=len,
    #is_separator_regex: 지정한 구분자가 정규식인지 여부를 지정하는 플래그
    is_separator_regex=False,
)
splitted_docs = recursive_text_splitter.split_documents(docs)

#이것은 CharacterTextSplitter: 텍스트를 일정한 길이나 규칙에 따라 분할하는 단순한 방법을 제공. 
#CharacterTextSplitter와 비슷한 방식으로 텍스트를 나누고, 
#나누어진 조각이 여전히 너무 길면 다시 재귀적으로 분할을 시도
#text_splitter = CharacterTextSplitter(
    #separator="\n\n", -> 텍스트 분할 시 사용할 구분자
    #chunk_size=100, -> 설명 완료
    #chunk_overlap=10,-> 설명 완료
    #length_function=len,-> 설명 완료
    #is_separator_regex=False,-> 설명 완료
#)
#splits = text_splitter.split_documents(docs)

In [14]:
#너무 길어서 여기는 주석처리로... enumerate을 통해 index와 각 청크를 따로 분리 후, 
#for i, splitted in enumerate(splitted_docs[:50]):
    #print(f'Chunk {i+1}:')
    #print(splitted)
    #print('=' * 50)

In [15]:
from langchain_openai import OpenAIEmbeddings
from uuid import uuid4
import faiss
from langchain.vectorstores import Chroma
embeddings = OpenAIEmbeddings(model = 'text-embedding-ada-002')

#from_documents를 통해 쉽게 vector_store 생성
vector_store = Chroma.from_documents(documents = splitted_docs, embedding = embeddings)
#심심하니 uuid4를 사용해 이미 chunk로 변한 값들을 고유한 코드로 된 값들로 다시 변경(이러면 PC가 사용하기 더 쉬워진다나 뭐라나...)
uuids = [str(uuid4()) for _ in range(len(splitted_docs))]
vector_store.add_documents(documents = splitted_docs, ids = uuids)

['93c3ca5a-879c-4fe9-8b2d-786cef50ba2c',
 '0dd23bc7-ba06-4268-ad7d-fb7d43cf16fe',
 '7bc6fe9e-0fed-478e-ad20-6bd2a692d241',
 '45877f0c-884c-4615-8b80-20d63c7bc868',
 'dec76e09-c2e3-4258-b1c0-54d47b8dc5fe',
 '8e84c01e-aba0-44dd-a658-9ec86508cfb5',
 'e0d9b153-0047-4131-8887-c15f8da08e64',
 '1d0d5f79-c82f-4305-9ee4-557e99df8186',
 'eff56d3e-fc70-479d-a98d-cec14b22d161',
 '6b5e0f89-df8f-4582-8f29-a69a8d595fc0',
 '4280faf4-de4d-4a62-a27d-0d7c43f17d9e',
 '5dee5749-674a-4b2a-9329-d554bd951d2c',
 'd4e1ebbb-d66e-4f93-a04d-52a252c7cf47',
 '5c2c8757-eae7-45b0-8f6d-50af75e0554b',
 '52b6c27c-b63d-4af9-9988-1904918b241f',
 'b8ce8366-8dcb-45e3-b644-8b31d4926b19',
 'c59ac49a-4ac1-49c2-9b8f-1b560efdd10e',
 'b7cbac15-1ca2-4dec-9139-1d91a273cac3',
 '02535324-7c2c-4751-a696-e8da298dbfae',
 '375607d3-06ee-420b-9774-b9889b4d52e3',
 '56809f28-dc89-4425-87f3-0c27285a1b72',
 '0d013851-08d1-4399-8d03-11d52a58a1ea',
 'fdcb9b98-b353-42b8-8981-0a50ce808143',
 '94b94d36-e638-4b7a-825c-6dd37862cb8c',
 'bf0b8f7c-40c2-

In [16]:
from langchain.chains import RetrievalQA
from langchain.chains import LLMChain
#여기서는 평병하게 retriever를 구성하였습니다.
#Chroma에서 이 부분 llm = client를 통해 쉽게 LLM에 연결이 가능합니다.
retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = {'k':5})
#여기서는 RetrieveralQA를 써서 질문과 답변을 받을 수 있게 만드는 함수
qa_retriever = RetrievalQA.from_chain_type(
    llm = client,
    chain_type = 'refine',
    retriever = retriever,
    return_source_documents = True,
)

In [17]:
import logging
#만약 질문을 계속 받을시에 작동할 while True: 하지만 비교를 위해 못 쓴다는...
#query로 답을 받아, 그것을 토대로 값을 출
#while True:
query = input('질문을 해주세요:')
#if query == 'exit':
    #print('좋은 대화였습니다!')
    #break
result = qa_retriever(query)
answer_a = result['result']
source_documents = result['source_documents']

    # 결과 출력
print("Answer:", answer_a)
#하지만 QA특성상 추출할 값도 가져와야 하는데, 값을 가져오고 띄우지는 못하게 하는 코드
logging.info("Source Documents: %s", source_documents)

질문을 해주세요: 초거대 언어모델의 성능향상을 위한 연구에 대해 알려줘


Answer: 초거대 언어모델의 성능 향상을 위한 연구는 다양한 측면에서 진행되고 있으며, 이는 언어모델의 발전 과정을 이해하는 데 중요한 배경을 제공합니다. ChatGPT와 같은 초거대 언어모델의 등장은 기존에 병렬적으로 연구되던 다양한 자연언어처리(NLP) 하위 분야를 하나의 모델로 처리할 수 있게 하면서, 태스크 수렴 현상을 가져왔습니다. 즉, 하나의 LLM을 통해 번역, 요약, 질의응답, 형태소 분석 등의 작업을 모두 처리할 수 있게 되었습니다.

이러한 발전은 주로 프롬프트를 어떻게 입력하느냐에 따라 모델의 다양한 능력이 창발되는 점에 기반하고 있습니다. 사용자는 프롬프트를 조정함으로써 모델이 특정 태스크에 맞는 출력을 생성하도록 할 수 있습니다. 이를 통해 LLM은 사용자의 목적에 맞는 다양한 출력을 생성할 수 있는 새로운 패러다임을 제공하고 있습니다.

또한, 데이터의 질과 양을 향상시키는 연구는 모델의 성능을 높이는 데 필수적입니다. 방대한 양의 데이터를 효과적으로 활용하기 위한 방법론이 개발되고 있으며, 이는 모델의 학습과 일반화 능력을 향상시키는 데 기여합니다. 최신 초거대 언어모델 연구 동향을 심층적으로 살펴보면, 새로운 평가 방식과 기준이 등장하고 있으며, 이는 모델의 다양한 언어적 과제 수행 능력을 측정하고 실제 응용 사례에서의 성능을 개선하는 데 도움을 줍니다.

마지막으로, 초거대 언어모델의 윤리적 사용과 관련된 연구도 중요하게 다루어지고 있습니다. 이는 모델이 편향되지 않도록 하고, 사용자에게 해를 끼치지 않도록 보장하는 방법을 찾는 것을 포함합니다. 이러한 연구들은 초거대 언어모델이 더 안전하고 책임감 있게 사용될 수 있도록 하는 데 기여하며, 연구자와 관련 전문가들에게 유용한 통찰과 지침을 제공합니다.

이와 같은 연구들은 초거대 언어모델의 전반적인 동향과 중요한 주제들에 대한 체계적인 이해를 제공하며, 자연언어처리의 발전과 함께 지속적으로 발전하고 있습니다. 이를 통해 연구자들은 더욱 향상된 모델을 개발하고, 다양한 언어적 과

In [18]:
#일반 LLM 모델인 GPT를 이용해 같은 질문을 받아 출력 비교
from openai import OpenAI
client2 = OpenAI()
completion = client2.chat.completions.create(
    model = 'gpt-4o',
    messages = [
        {'role' : 'system', 'content' : '넌 초거대 언어모델 연구 동향에 대해 설명해주는 AI야. 여러 관련 모델들과 지식들을 설명해줘.'},
        {'role' : 'user', 'content' : 'query'}
    ]
)
answer_b = completion.choices[0].message.content
print(answer_b)

초거대 언어 모델(Giant Language Model)은 대규모 자연어 처리(NLP) 문제를 해결하기 위해 개발된 인공지능 모델입니다. 이러한 모델들은 방대한 양의 텍스트 데이터를 학습하여 인간과 유사한 방식으로 언어를 이해하고 생성할 수 있도록 설계되었습니다. 여기에는 문서 요약, 번역, 질문 응답, 대화 생성 등의 다양한 작업이 포함됩니다. 최근 몇 년간 주목할 만한 초거대 언어 모델에는 다음과 같은 것들이 있습니다.

1. **GPT 시리즈**: 오픈AI가 개발한 모델로서, 다양한 언어 처리 작업에서 혁신적인 성능을 보여주었습니다. 특히 GPT-3와 GPT-4는 각각 155억 개와 수천억 개의 매개변수를 가진 모델로, 자연어 생성에서 뛰어난 능력을 보입니다.

2. **BERT 및 확장 버전**: 구글이 개발한 BERT는 양방향 트랜스포머 기반의 언어 모델입니다. 이후 여러 확장 모델이 개발되어, BERT의 성능을 향상시키거나 특정 작업에 맞추어 특화된 형태로 발전하였습니다.

3. **T5 (Text-to-Text Transfer Transformer)**: 이 모델은 대부분의 NLP 작업을 텍스트 입력을 텍스트 출력으로 변환하는 일관된 접근 방식으로 통합하여 수행합니다. 이 모델은 다양한 언어 이해 및 생성 작업에서 뛰어난 성과를 보여줍니다.

4. **LaMDA (Language Model for Dialogue Applications)**: 구글이 개발한 대화용 언어 모델로, 인간과 같은 대화를 생성하는 데 특화되어 있습니다.

5. **Claude**: Anthropic에서 개발한 모델로, 인간 중심의 AI 안전성을 중점으로 개발되었습니다. 대규모 언어 모델을 개발하는 데 있어 윤리와 안전성을 강조합니다.

6. **LLaMA (Large Language Model Meta AI)**: Meta AI에서 개발한 모델로, 연구 공동체가 쉽게 접근할 수 있도록 공개되었습니다. 다양한 크기의 버전이 존재하며, 연구 및 사용자 지정 작업에 사용됩니다.

In [19]:
#A가 더 좋다는 답변을 받았다! 즉, 내가 만든 값이 더 정확하고 좋다는 뜻!
prompt = f"""[System]
Please act as an impartial judge and evaluate the quality of the responses provided by two
AI assistants to the user question displayed below. You should choose the assistant that
follows the user's instructions and answers the user's question better. Your evaluation
should consider factors such as the helpfulness, relevance, accuracy, depth, creativity,
and level of detail of their responses. Begin your evaluation by comparing the two
responses and provide a short explanation. Avoid any position biases and ensure that the
order in which the responses were presented does not influence your decision. Do not allow
the length of the responses to influence your evaluation. Do not favor certain names of
the assistants. Be as objective as possible. After providing your explanation, output your
final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]"
if assistant B is better, and "[[C]]" for a tie.

[User Question]
{query}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]"""

completion = client2.chat.completions.create(
    model = 'gpt-4o',
    messages = [
        {'role' : 'user', 'content' : prompt}
    ]
)
final = completion.choices[0].message.content
print(final)

Both Assistant A and Assistant B provide informative answers about the research efforts to enhance the performance of large language models. However, they focus on different aspects of the topic.

Assistant A offers a detailed overview of the research aimed at improving the performance of large language models, discussing the integration of different NLP tasks into a single model, the role of prompt engineering, improvements in data quality and quantity, new evaluation methods, and ethical considerations. The response covers various dimensions of research, demonstrating an understanding of the complexity and scope of advancements in large language models. The discussion on prompt engineering, data methodologies, and ethical research adds depth to the response.

Assistant B, on the other hand, provides a brief overview of several prominent language models, including GPT, BERT, T5, LaMDA, Claude, and LLaMA. The response also touches on recent research trends, including social impact, eth

In [20]:
"""RAG 모델은 먼저 입력된 질문과 관련된 정보를 검색하고, 그 정보를 바탕으로 답변을 생성합니다. 
이렇게 하면 모델이 정확하고 최신의 정보를 기반으로 답변을 생성할 수 있습니다.
특히 Chroma와 같은 벡터 데이터베이스를 사용하여 RAG 모델을 구현할 때, 모델의 성능을 크게 향상시킬 수 있습니다.
특히 Chroma는 벡터 데이터베이스로, 문서나 텍스트 데이터를 벡터화하여 빠르고 효율적인 검색을 지원하는데, RAG 모델에서 Chroma와 같은 검색 시스템을 사용하면, 
모델이 실시간으로 관련 정보를 빠르게 검색하고, 그 정보를 바탕으로 더 적합한 답변을 생성할 수 있습니다.
한 마디로 LLM 같은 대규모 랭귀지 모델에서 RAG를 통해 최신 정보를 토대로 정확한 답을 생성해낼 수 있기 때문입니다."""

'RAG 모델은 먼저 입력된 질문과 관련된 정보를 검색하고, 그 정보를 바탕으로 답변을 생성합니다. \n이렇게 하면 모델이 정확하고 최신의 정보를 기반으로 답변을 생성할 수 있습니다.\n특히 Chroma와 같은 벡터 데이터베이스를 사용하여 RAG 모델을 구현할 때, 모델의 성능을 크게 향상시킬 수 있습니다.\n특히 Chroma는 벡터 데이터베이스로, 문서나 텍스트 데이터를 벡터화하여 빠르고 효율적인 검색을 지원하는데, RAG 모델에서 Chroma와 같은 검색 시스템을 사용하면, \n모델이 실시간으로 관련 정보를 빠르게 검색하고, 그 정보를 바탕으로 더 적합한 답변을 생성할 수 있습니다.\n한 마디로 LLM 같은 대규모 랭귀지 모델에서 RAG를 통해 최신 정보를 토대로 정확한 답을 생성해낼 수 있기 때문입니다.'

In [21]:
from rouge import Rouge
rouge = Rouge()
score = rouge.get_scores(answer_a, answer_b)
print(score)

[{'rouge-1': {'r': 0.14432989690721648, 'p': 0.1590909090909091, 'f': 0.15135134636318495}, 'rouge-2': {'r': 0.01652892561983471, 'p': 0.018018018018018018, 'f': 0.01724137431963581}, 'rouge-l': {'r': 0.13917525773195877, 'p': 0.1534090909090909, 'f': 0.14594594095777957}}]
